# Testes de treinamento tamanho de embeddings

A ideia deste notebook é verificar se treinar os algoritmos de MAB durante as interações de teste geram alguma melhora para eles, comparando-os com os resultados de um algoritmo estático (ALS).

In [ ]:
import pandas as pd
from mab2rec import BanditRecommender, LearningPolicy
from mab2rec.pipeline import train
from sklearn.preprocessing import LabelEncoder
import implicit
from scipy.sparse import csr_matrix
from implicit.nearest_neighbours import bm25_weight
import plotly.express as px
import time
import os
import math

train_data = "../data/ml100k/data_train.csv"
test_data = "../data/ml100k/data_test.csv"

In [ ]:
def initial_train(df, num_users, num_items, factors):
    K1 = 100
    B = 0.8

    # Cria a matriz esparsa
    sparse_matrix = csr_matrix((df['response'], (df['user_id'], df['item_id'])), shape=(num_users, num_items))
    updated_sparse_matrix = bm25_weight(sparse_matrix, K1=K1, B=B)

    print('Treinando o modelo ALS')
    ALS_model = implicit.als.AlternatingLeastSquares(factors=factors, random_state=1)
    ALS_model.fit(updated_sparse_matrix)

    user_features_list = []

    for user_id in df['user_id'].unique():
        user_factors = ALS_model.user_factors[user_id]
        user_features_list.append([user_id] + list(user_factors))

    df_user_features = pd.DataFrame(user_features_list, columns=['user_id'] + [f'u{i}' for i in range(factors)])

    return ALS_model,  df_user_features, sparse_matrix

In [ ]:

def test_ALS(ALS_model, sparse_matrix, df_test):
    print('Testing ALS')
    all_recs = []

    start_time = time.time()
    hits = 0
    for i, interaction in df_test.iterrows():
        ids_recs, _ = ALS_model.recommend(userid=interaction['user_id'], user_items=sparse_matrix[interaction['user_id']], N=10)
        if interaction['item_id'] in ids_recs:
            hits += 1
        all_recs.append(ids_recs.tolist())
    
    recs_df = pd.DataFrame({
        'interaction_number': [i for i in range(len(df_test))],
        'user_id': df_test['user_id'],
        'item_id': df_test['item_id'],
        'recommendations': all_recs
    })
    

    return hits, hits/len(df_test), time.time() - start_time, recs_df

In [ ]:
def test_non_incremental(mab_algo, algo_name, user_features, df_test):
    start_time = time.time()
    hits = 0

    contexts = df_test.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

    recomendations = mab_algo.recommend(contexts)

    df_test = df_test.reset_index(drop=True)

    hits = 0
    for i, interaction in df_test.iterrows():
        if interaction['item_id'] in recomendations[i]:
            hits += 1
    
    recs_df = pd.DataFrame({
        'interaction_number': [i for i in range(len(df_test))],
        'user_id': df_test['user_id'],
        'item_id': df_test['item_id'],
        'recommendations': recomendations
    })

    return hits, hits/len(df_test), time.time() - start_time, recs_df

In [ ]:
def test_incremental(mab_algo, algo_name, user_features, df_test, df_test_for_evaluation, batch_size):
    recs = []

    start_time = time.time()
    hits = 0

    for i in range(0, len(df_test), batch_size):
        # Fazendo recomendações para teste
        df_batch_test = df_test_for_evaluation.loc[i:i+batch_size-1]
        contexts = df_batch_test.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

        if len(contexts) > 0: # Se não tiver nenhuma interação positiva, não faz sentido fazer recomendações
            recomendations = mab_algo.recommend(contexts)
            if isinstance(recomendations, list) and isinstance(recomendations[0], int):
                # Quando o contexto tem tamanho 1, a recomendação é uma lista, e não uma lista de listas
                recomendations = [recomendations]

            df_batch_test = df_batch_test.reset_index(drop=True)

            for j, interaction in df_batch_test.iterrows():
                if interaction['item_id'] in recomendations[j]:
                    hits += 1
            
            recs.extend(recomendations)
        
        # Treinando com o batch
        df_batch_train = df_test.loc[i:i+batch_size-1]
        contexts = df_batch_train.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

        mab_algo.partial_fit(df_batch_train['item_id'], df_batch_train['response'], contexts)
    
    recs_df = pd.DataFrame({
        'interaction_number': [i for i in range(len(df_test_for_evaluation))],
        'user_id': df_test_for_evaluation['user_id'],
        'item_id': df_test_for_evaluation['item_id'],
        'recommendations': recs
    })

    return hits, hits/len(df_test_for_evaluation), time.time() - start_time, recs_df

In [ ]:
def test(train_size, factors_list):
    save_path = f'results-v3/{round(train_size * 100):02}-{round((1 - train_size) * 100):02}'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    results = []
    df_recs = pd.DataFrame(columns=['algorithm', 'hiperparams', 'batch_size', 'interaction_number', 'user_id', 'item_id', 'recommendations'])
    df_train = pd.read_csv(train_data)
    df_test = pd.read_csv(test_data)

    df_full = pd.concat([df_train, df_test])

    df_full['user_id'] = LabelEncoder().fit_transform(df_full['user_id'])
    df_full['item_id'] = LabelEncoder().fit_transform(df_full['item_id'])

    num_users = df_full['user_id'].nunique()
    num_items = df_full['item_id'].nunique()

    split_index = int(len(df_full) * train_size)
    df_train = df_full[:split_index]
    df_test = df_full[split_index:]

    df_test = df_test[(df_test['user_id'].isin(df_train['user_id'])) & (df_test['item_id'].isin(df_train['item_id']))]
    df_test = df_test.reset_index(drop=True)
    df_test_for_evaluation = df_test[df_test['response'] == 1]

    for factors in factors_list:
        print(f'Treinando com {factors} fatores')

        ALS_model, df_user_features, sparse_matrix = initial_train(df_train, num_users, num_items, factors)

        hits, hr, spent_time, df_recs_als = test_ALS(ALS_model, sparse_matrix, df_test_for_evaluation)
        df_recs_als['algorithm'] = 'ALS'
        df_recs_als['factors'] = [factors for _ in range(len(df_recs_als))]
        df_recs = pd.concat([df_recs, df_recs_als])
        results.append({'algorithm': 'ALS', 'hits': hits, 'hr': hr, 'time': spent_time, 'factors': factors})

        print('\nTesting LinUCB non-incremental\n')

        linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(alpha=0.1), top_k=10)
        train(linUCB_model, data=df_train, user_features=df_user_features)

        hits, hr, spent_time, df_recs_linUCB = test_non_incremental(linUCB_model, 'LinUCB', df_user_features, df_test_for_evaluation)
        df_recs_linUCB['algorithm'] = 'LinUCB (non_incremental)'
        df_recs_linUCB['factors'] = [factors for _ in range(len(df_recs_linUCB))]
        df_recs = pd.concat([df_recs, df_recs_linUCB])
        results.append({'algorithm': 'LinUCB (non_incremental)', 'hits': hits, 'hr': hr, 'time': spent_time, 'factors': factors})

        print('\nTesting LinGreedy non-incremental\n')

        linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(epsilon=0.01), top_k=10)
        train(linGreedy_model, data=df_train, user_features=df_user_features)

        hits, hr, spent_time, df_recs_linGreedy = test_non_incremental(linGreedy_model, 'LinGreedy', df_user_features, df_test_for_evaluation)
        df_recs_linGreedy['algorithm'] = 'LinGreedy (non_incremental)'
        df_recs_linGreedy['factors'] = [factors for _ in range(len(df_recs_linGreedy))]
        df_recs = pd.concat([df_recs, df_recs_linGreedy])
        results.append({'algorithm': 'LinGreedy (non_incremental)', 'hits': hits, 'hr': hr, 'time': spent_time, 'factors': factors})

        print('\nTesting LinUCB incremental\n')

        linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(alpha=0.1), top_k=10)
        train(linUCB_model, data=df_train, user_features=df_user_features)

        hits, hr, spent_time, df_recs_linUCB = test_incremental(linUCB_model, 'LinUCB', df_user_features, df_test, df_test_for_evaluation, 10)
        df_recs_linUCB['algorithm'] = 'LinUCB (incremental)'
        df_recs_linUCB['factors'] = [factors for _ in range(len(df_recs_linUCB))]
        df_recs = pd.concat([df_recs, df_recs_linUCB])
        results.append({'algorithm': 'LinUCB (incremental)', 'hits': hits, 'hr': hr, 'time': spent_time, 'factors': factors})

        print('\nTesting LinGreedy incremental\n')

        linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(epsilon=0.01), top_k=10)
        train(linGreedy_model, data=df_train, user_features=df_user_features)

        hits, hr, spent_time, df_recs_linGreedy = test_incremental(linGreedy_model, 'LinGreedy', df_user_features, df_test, df_test_for_evaluation, 10)
        df_recs_linGreedy['algorithm'] = 'LinGreedy (incremental)'
        df_recs_linGreedy['factors'] = [factors for _ in range(len(df_recs_linGreedy))]
        df_recs = pd.concat([df_recs, df_recs_linGreedy])
        results.append({'algorithm': 'LinGreedy (incremental)', 'hits': hits, 'hr': hr, 'time': spent_time, 'factors': factors})

    print('\nTesting Random\n')

    random_model = BanditRecommender(learning_policy=LearningPolicy.Random(), top_k=10)
    train(random_model, data=df_train, user_features=df_user_features)

    hits, hr, spent_time, df_recs_random = test_non_incremental(random_model, 'Random', df_user_features, df_test_for_evaluation)
    df_recs_random['algorithm'] = 'Random'
    df_recs = pd.concat([df_recs, df_recs_random])
    results.append({'algorithm': 'Random', 'hits': hits, 'hr': hr, 'time': spent_time})
    
    df_results = pd.DataFrame(results)
    df_results = df_results.astype({'hits': int, 'hr': float, 'time': float})
    df_results['test_size'] = round(1 - train_size, 2)
    df_results['test_interactions'] = len(df_test_for_evaluation)

    df_results.to_csv(f'{save_path}/results.csv', index=False)
    df_recs.to_csv(f'{save_path}/recs.csv', index=False)

In [ ]:
train_sizes = [0.95, 0.90, 0.85, 0.80]
factors_list = [2, 4, 8, 16, 32, 64, 128, 256, 512]

for train_size in train_sizes:
    test(
        train_size=train_size, 
        factors_list=factors_list
    )

In [43]:
train_sizes = [0.95, 0.90, 0.85, 0.80]
all_dfs = []

for train_size in train_sizes:
    save_path = f'results-v3/{round(train_size * 100):02}-{round((1 - train_size) * 100):02}/results.csv'
    all_dfs.append(pd.read_csv(save_path))
    
df_results = pd.concat(all_dfs)
df_results

,algorithm,hits,hr,time,factors,test_size,test_interactions
0,ALS,36,0.120401,0.290088,2.0,0.05,299
1,LinUCB (non_incremental),23,0.076923,0.094181,2.0,0.05,299
2,LinGreedy (non_incremental),21,0.070234,0.076130,2.0,0.05,299
3,LinUCB (incremental),35,0.117057,6.880607,2.0,0.05,299
4,LinGreedy (incremental),35,0.117057,6.302572,2.0,0.05,299
...,...,...,...,...,...,...,...
41,LinUCB (non_incremental),108,0.067332,4.479172,512.0,0.20,1604
42,LinGreedy (non_incremental),127,0.079177,0.775431,512.0,0.20,1604
43,LinUCB (incremental),33,0.020574,569.399015,512.0,0.20,1604
44,LinGreedy (incremental),29,0.018080,522.678063,512.0,0.20,1604


In [44]:
df_results.to_csv('results-v3/concat_results.csv', index=False)

In [45]:
df_results_new = df_results.copy()
df_results_new['algorithm'] = df_results_new['algorithm'] + ' - ' + df_results_new['factors'].astype(str)

fig = px.line(df_results_new, x="test_size", y="hr", color='algorithm', title='HR x Test size')
fig.show()
fig.write_html('results-v3/hr_x_test_size.html')

In [46]:
algo_name = 'LinGreedy (incremental)'  # Escolha um algoritmo específico para visualizar

df_results_filtered = df_results[(df_results['algorithm'] == algo_name) | (df_results['algorithm'] == 'Random')]
df_results_filtered['algorithm'] = df_results_filtered['algorithm'] + ' - ' + df_results_filtered['factors'].astype(str)

fig = px.line(df_results_filtered, x="test_size", y="hr", color='algorithm', title='HR x Test size')
fig.show()

C:\Users\gregf\AppData\Local\Temp\ipykernel_26344\2201864770.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

